# Historical Tycho

Notebook criado para tratar os dados do corpus histórico do português brasileiros Tycho Brahe
O objetivo deste notebook é tratar o corpus para gerar um modelos de Word Embeddings para diferentes gerações e, posteriormente, criar uma representação gráfica da mudança semântica para o português brasileiro

In [ ]:
# importanto bibliotecas utilizadas
import os, shutil, re

In [ ]:
# Biblioteca criada para tratar os dados
import corpus_man as cm

In [ ]:
nlp = cm.spacy_loader()
nlp.Defaults.stop_words |= {"character",'comment','title','author','language',}
nlp.disable_pipes('ner')

In [ ]:
# Mostrando a remoção do Reconhecimento de Entidades Nomeadas para agilizar o processo
nlp.pipeline


## Preparação do Corpus

Removendo os cabeçalhos dos arquivos .txt, transformando os tokens em lowercase e retirando stopwords e acentos

### Separação em sentenças

In [ ]:
if not os.path.exists('./txt/sentencized'):
    os.makedirs('./txt/sentencized')

In [ ]:
for root, dirs, files in os.walk("./txt/original", topdown=False):
    for name in files:
        with open(os.path.join(root,name),'r') as file:
            txt = file.read()
            txt = re.split(r'\[END HEADER\]\W+',txt)[1]
            doc = nlp(txt)
            with open('./txt/sentencized/'+ name, 'w') as sentencized:
                for sent in doc.sents:
                    sentencized.write(str(sent))
                    sentencized.write('\n')


### Remoção de quebras de linhas em excesso

### Lematização, remoção de acentos e lowercase

In [ ]:
for root, dirs, files in os.walk("./txt/sentencized", topdown=False):
    for name in files:
        with open(os.path.join(root,name),'r') as file:
            with open('./txt/lematized_lookup/'+ name, 'w') as processed:
                for line in file.readlines():
                    if line.strip():
                        processed.write(cm.pre_process(line))
                        processed.write('\n')


## Classificando os arquivos

O próximo passo é ordenar e agrupar os arquivos de texto de acordo com a metainformação "Period by Birthdate", que corresponde ao período do texto de acordo com a data de nascimento do autor. Os textos foram manualmente classificados de acordo com seu pertencimento a cada período como definido por "Bechara"

In [ ]:
import csv, shutil

In [ ]:
cor_dict = {}
with open('periodos_tycho.csv', mode='r') as file:
    reader = csv.reader(file)
    cor_dict = {row[1]:row[0] for row in reader}

A variável dir_path contém o caminho até a pasta de destino onde os textos serão separados em períodos diferentes.
Para a criação de diferentes versões de modelos somente esse caminho foi alterado.

In [ ]:
dir_path = 'txt_sorts/v2'
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

if not os.path.exists(dir_path + '/p1'):
    os.makedirs(dir_path + '/p1')
if not os.path.exists(dir_path + '/p2'):
    os.makedirs(dir_path + '/p2')
if not os.path.exists(dir_path + '/p3'):
    os.makedirs(dir_path + '/p3')    

In [ ]:
for root, dirs, files in os.walk("./txt/processed", topdown=False):
    for file in files:
        if cor_dict[file] == 'arcmed':
            source = os.path.join(root,file)
            target = dir_path + '/p1'
            shutil.copy(source,target)
        if cor_dict[file] == 'mod':
            source = os.path.join(root,file)
            target = dir_path + '/p2'
            shutil.copy(source,target)
        if cor_dict[file] == 'cont1' or cor_dict[file] == 'cont2':
            source = os.path.join(root,file)
            target = dir_path + '/p3'
            shutil.copy(source,target)

### Criando um dump

In [ ]:
dumps_path = dir_path+'/dumps'

if not os.path.exists(dumps_path):
    os.makedirs(dumps_path)

In [ ]:
with open(dumps_path+'/dump1', 'w') as dump:
    for root, dirs, files in os.walk(dir_path+'/p1', topdown=False):
        for file in files:
            with open(os.path.join(root,file),'r') as text:
                for content in text.readlines():
                    dump.write(content)
                

In [ ]:
with open(dumps_path+'/dump2', 'w') as dump:
    for root, dirs, files in os.walk(dir_path+'/p2', topdown=False):
        for file in files:
            with open(os.path.join(root,file),'r') as text:
                for content in text.readlines():
                    dump.write(content)


In [ ]:
with open(dumps_path+'/dump3', 'w') as dump:
    for root, dirs, files in os.walk(dir_path+'/p3', topdown=False):
        for file in files:
            with open(os.path.join(root,file),'r') as text:
                for content in text.readlines():
                    dump.write(content)

## Treinamento dos Modelos

Os modelos foram treinados serparadamente usando o código fornecido por [Mikolov et al ](https://github.com/tmikolov/word2vec).

Os hiperparâmetros utilizados foram:  
-cbow 1  
-size 300   
-window 8   
-negative 25   
-hs 0  
-sample 1e-4   
-threads 20   
-binary 1   
-iter 25   

# Carregando modelos do Gensim

In [ ]:
import gensim

Os modelos foram carregados um a um separadamente e em seguida plotados utilizando a biblioteca plotly

In [ ]:
dump = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format(
    './txt_sorts/v2/vectors/vectors_dump1.bin',
    binary=True)
# dump = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format(
#     './txt_sorts/v2/vectors/vectors_dump2.bin',
#     binary=True)
# dump = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format(
#     './txt_sorts/v2/vectors/vectors_dump3.bin',
#     binary=True)

In [ ]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling


def reduce_dimensions(model):
    '''Função para reduzir o número de dimensões dos vetores de palavras a 2.
    model : gensim.Keyedvector
    '''
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    # extract the words & their vectors, as numpy arrays
    vectors = np.asarray(model.vectors)
    labels = np.asarray(model.index_to_key)  # fixed-width numpy strings

    # reduce using t-SNE
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

def my_labels(x_val, y_val, labels, my_words):
    '''Função para recuperar os índices de palavras a serem destacadas na visualização
    my_words : list of strings
    labels : np.arrey
    y_val : list
    x_val : list
    '''
    inxs = [np.where(labels == i)[0][0] for i in my_words]    
    return ([x_val[i] for i in inxs], [y_val[i] for i in inxs])

# Usando a função para obter valores de x, y e os rótulos para as coordenadas
x_vals, y_vals, labels = reduce_dimensions(dump)

In [ ]:
my_words = ['deus','mulher','homem','pai','mae','terra']

my_x,my_y = my_labels(x_vals,y_vals,labels, my_words)

In [ ]:
def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go
    
    fig = go.Figure()
    fig.add_trace(go.Scattergl(x=x_vals, y=y_vals, mode='text', text=labels,
                              ))
    fig.add_trace(go.Scattergl(x=my_x,y=my_y,mode='text',text=my_words,
                               marker_color='red'))
    fig.for_each_trace(lambda t: t.update(textfont_color=t.marker.color, textposition='top center'))
    fig.show()


plot_with_plotly(x_vals, y_vals, labels)